<a href="https://colab.research.google.com/github/YouJoohyung/myfirst-project/blob/master/PR_03_01_bayesian_keras_hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Module**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print (tf.__version__)
print (tf.keras.__version__)

2.8.0
2.8.0


##install bayesian-optimization

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=8ada55308b9608bf07a7cef540a6102773f7384a176c0a1d2cad56e851e848f5
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


## load dataset

In [ ]:
NUM_CLASSES = 10

def get_input_datasets(use_bfloat16=False):
    # input image dimensions
    img_rows, img_cols = 28, 28
    cast_dtype = tf.bfloat16 if use_bfloat16 else tf.float32

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    if tf.keras.backend.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    # convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

    # train dataset
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.repeat()
    train_ds = train_ds.map(lambda x, y: (tf.cast(x, cast_dtype), y))
    train_ds = train_ds.batch(64, drop_remainder=True)

    # eval dataset
    eval_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    eval_ds = eval_ds.repeat()
    eval_ds = eval_ds.map(lambda x, y: (tf.cast(x, cast_dtype), y))
    eval_ds = eval_ds.batch(64, drop_remainder=True)

    return train_ds, eval_ds, input_shape

##define model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, Flatten, Activation
from tensorflow.keras.optimizers import RMSprop

def get_model(input_shape, dropout2_rate=0.5):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape,
                     name="conv2d_1"))
    model.add(Conv2D(64, (3, 3), activation='relu', name="conv2d_2"))
    model.add(MaxPooling2D(pool_size=(2, 2), name="maxpool2d_1"))
    model.add(Dropout(0.25, name="dropout_1"))
    model.add(Flatten(name="flatten"))
    model.add(Dense(128, activation='relu', name="dense_1"))
    model.add(Dropout(dropout2_rate, name="dropout_2"))
    model.add(Dense(NUM_CLASSES, activation='softmax', name="dense_2"))
    return model

In [ ]:
train_ds, eval_ds, input_shape = get_input_datasets()

print (input_shape)

11501568/11490434 [==============================] - 0s 0us/step
(28, 28, 1)


##fit model

In [ ]:
def fit_with(input_shape, verbose, dropout2_rate, lr):

    # create the model using a specified hyperparameters.
    model = get_model(input_shape, dropout2_rate)

    # train the model for a specified number of epochs.
    optimizer = RMSprop(learning_rate=lr)

    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # train the model with the train dataset.
    model.fit(x=train_ds, epochs=1, steps_per_epoch=468, verbose=verbose)
    
    # evaluate the model with the eval dataset.
    score = model.evaluate(eval_ds, steps=10, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # return the accuracy.
    return score[1]

In [ ]:
from functools import partial

verbose = 1
fit_with_partial = partial(fit_with, input_shape, verbose)

In [ ]:
#fit_with_partial(dropout2_rate=0.5, lr=0.001)

In [ ]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'dropout2_rate': (0.1, 0.5), 'lr': (1e-4, 1e-2)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=10,)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))


|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------
468/468 [==============================] - 88s 186ms/step - loss: 0.3248 - accuracy: 0.9151
Test loss: 0.06753873080015182
Test accuracy: 0.9828125238418579
|  1        |  0.9828   |  0.2668   |  0.007231 |
468/468 [==============================] - 76s 161ms/step - loss: 0.1784 - accuracy: 0.9456
Test loss: 0.05733022838830948
Test accuracy: 0.9859374761581421
|  2        |  0.9859   |  0.1      |  0.003093 |
468/468 [==============================] - 76s 162ms/step - loss: 0.1958 - accuracy: 0.9410
Test loss: 0.04921071603894234
Test accuracy: 0.9859374761581421
|  3        |  0.9859   |  0.1587   |  0.001014 |
468/468 [==============================] - 76s 160ms/step - loss: 0.1919 - accuracy: 0.9429
Test loss: 0.049167562276124954
Test accuracy: 0.9906250238418579
|  4        |  0.9906   |  0.1745   |  0.003521 |
468/468 [==============================] - 75s 158ms/step - loss: 0.215

KeyboardInterrupt: ignored

In [ ]:
print (optimizer.max)

{'target': 0.989062488079071, 'params': {'dropout2_rate': 0.10004574992693796, 'lr': 0.003093092469055214}}


In [ ]:
optimizer.probe(
    params={"dropout2_rate": (0.3), "lr": (1e-3)},
    lazy=True,
)

# Will probe only the two points specified above
optimizer.maximize(init_points=0, n_iter=0)

|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------
468/468 [==============================] - 3s 6ms/step - loss: 0.4471 - accuracy: 0.8583
Test loss: 0.05281718820333481
Test accuracy: 0.984375
|  21       |  0.9844   |  0.3      |  0.001    |


In [ ]:
from bayes_opt import JSONLogger
from bayes_opt.event import Events

In [ ]:
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
# Results will be saved in ./logs.json
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------
468/468 [==============================] - 3s 7ms/step - loss: 0.7505 - accuracy: 0.8154
Test loss: 0.0678020566701889
Test accuracy: 0.9703124761581421
|  22       |  0.9703   |  0.4203   |  0.009686 |
468/468 [==============================] - 3s 6ms/step - loss: 0.5401 - accuracy: 0.8571
Test loss: 0.08527837693691254
Test accuracy: 0.9703124761581421
|  23       |  0.9703   |  0.2254   |  0.006954 |
468/468 [==============================] - 4s 7ms/step - loss: 0.4145 - accuracy: 0.8799
Test loss: 0.04319725185632706
Test accuracy: 0.987500011920929
|  24       |  0.9875   |  0.1001   |  0.002976 |
468/468 [==============================] - 4s 8ms/step - loss: 0.3792 - accuracy: 0.8820
Test loss: 0.052598170936107635
Test accuracy: 0.981249988079071
|  25       |  0.9812   |  0.1562   |  0.001957 |
468/468 [==============================] - 3s 6ms/step - loss: 0.7894 - accuracy: 0.80

In [ ]:
from bayes_opt.util import load_logs

new_optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

# New optimizer is loaded with previously seen points
load_logs(new_optimizer, logs=["./logs.json"]);

In [ ]:
print (new_optimizer.max)

{'target': 0.987500011920929, 'params': {'dropout2_rate': 0.10005666430688254, 'lr': 0.0029759677365857394}}
